# Step 1: Import Necessary Libraries

In [39]:
import geopandas as gpd
import json

# Step 2: Load Cleaned Data

In [41]:
# Load cleaned GeoJSON data
state_boundaries = gpd.read_file("state_geojson.geojson")
lga_boundaries = gpd.read_file("lga_geojson.geojson")
ward_boundaries = gpd.read_file("ward_geojson.geojson")

# Step 3: Create State-to-LGA and LGA-to-Ward Relationships

In [42]:
# Create mappings
state_to_lga = lga_boundaries.groupby("statename")["lganame"].apply(list).to_dict()
lga_to_ward = ward_boundaries.groupby("lganame")["wardname"].apply(list).to_dict()

# Generate HTML with full state names
state_options = "".join([f'<option value="{state}">{state}</option>' for state in state_to_lga.keys()])

# Step 4: Convert GeoDataFrames to GeoJSON

In [43]:
# Convert to GeoJSON format
state_geojson = state_boundaries.to_crs(epsg=4326).to_json()
lga_geojson = lga_boundaries.to_crs(epsg=4326).to_json()
ward_geojson = ward_boundaries.to_crs(epsg=4326).to_json()


# Step 5: Generate HTML for Interactive Map

In [54]:
html_template = f"""
<!DOCTYPE html>
<html>
<head>
    <meta charset="UTF-8" />
    <meta name="viewport" content="width=device-width, initial-scale=1.0" />
    <link rel="stylesheet" href="https://cdn.jsdelivr.net/npm/leaflet@1.9.3/dist/leaflet.css" />
    <link rel="stylesheet" href="https://cdn.jsdelivr.net/npm/bootstrap@5.2.2/dist/css/bootstrap.min.css" />
    <title>Interactive Map</title>
    <style>
        html, body {{
            width: 100%;
            height: 100%;
            margin: 0;
            padding: 0;
            font-family: Arial, sans-serif;
        }}
        #container {{
            display: flex;
            flex-direction: row;
            width: 100%;
            height: 100%;
        }}
        #filters {{
            width: 20%;
            padding: 20px;
            background: white;
            border-right: 1px solid #ddd;
            box-shadow: 2px 0 5px rgba(0, 0, 0, 0.1);
            overflow-y: auto;
        }}
        #filters h5 {{
            margin-bottom: 20px;
            font-size: 18px;
        }}
        #filters h6 {{
            margin-bottom: 10px;
            font-size: 16px;
        }}
        #filters select {{
            margin-bottom: 15px;
            width: 100%;
            padding: 5px;
        }}
        #download-button {{
            display: block;
            margin: 20px auto;
            width: 80%;
            background-color: #007bff;
            color: white;
            border: none;
            border-radius: 5px;
            padding: 10px;
            text-align: center;
            cursor: pointer;
        }}
        #map-container {{
            flex: 1;
            position: relative;
        }}
        #map {{
            height: 100%;
            width: 100%;
        }}
        #search-box {{
            position: absolute;
            top: 10px;
            right: 10px;
            z-index: 1000;
            width: 250px;
        }}
        .north-arrow {{
            position: absolute;
            top: 10px;
            left: 10px;
            z-index: 1000;
            background: white;
            border: 1px solid #ccc;
            padding: 5px;
        }}
        .state-label, .lga-label, .ward-label {{
            font-size: 18px;
            font-weight: bold;
            text-align: center;
            color: black;
            background: none;
            border: none;
            text-shadow: 0 0 50px rgba(128, 128, 128, 0.5);
            padding: 0;
            margin: 0;
            pointer-events: none;
        }}
        .leaflet-tooltip {{
            background: none;
            border: none;
            box-shadow: none;
        }}
    </style>
</head>
<body>
    <div id="container">
        <div id="filters">
            <h5>Filter Map</h5>
            <h6>State</h6>
            <select id="stateDropdown" class="form-select" multiple onchange="handleStateSelection()"></select>
            <h6>Local Government</h6>
            <select id="lgaDropdown" class="form-select" multiple onchange="handleLGASelection()"></select>
            <h6>Ward</h6>
            <select id="wardDropdown" class="form-select" multiple onchange="handleWardSelection()"></select>
            <button id="download-button" onclick="downloadMap()">Download Map</button>
        </div>
        <div id="map-container">
            <div id="search-box">
                <input
                    type="text"
                    id="searchInput"
                    class="form-control"
                    placeholder="Search State, LGA, or Ward"
                    oninput="handleSearch()"
                />
            </div>
            <div id="map"></div>
        </div>
    </div>

    <script src="https://cdn.jsdelivr.net/npm/leaflet@1.9.3/dist/leaflet.js"></script>
    <script src="https://cdnjs.cloudflare.com/ajax/libs/leaflet-image/0.4.0/leaflet-image.min.js"></script>
    <script src="https://cdnjs.cloudflare.com/ajax/libs/html2canvas/1.4.1/html2canvas.min.js"></script>
    <script>
        const stateGeoJSON = {state_geojson};
        const lgaGeoJSON = {lga_geojson};
        const wardGeoJSON = {ward_geojson};
        const stateToLGA = {json.dumps(state_to_lga)};
        const lgaToWard = {json.dumps(lga_to_ward)};

        const map = L.map('map').setView([9.0820, 8.6753], 6);
        L.control.scale().addTo(map);

        const northArrow = L.control({{ position: 'topright' }});
        northArrow.onAdd = function () {{
            const div = L.DomUtil.create('div', 'north-arrow');
            div.innerHTML = '↑<br>N';
            return div;
        }};
        northArrow.addTo(map);

        let stateLayer, lgaLayer, wardLayer;

        stateLayer = L.geoJSON(stateGeoJSON, {{
            style: function() {{ return {{ color: "#333333", weight: 1, fillOpacity: 0.5, fillColor: "beige" }}; }},
            onEachFeature: function(feature, layer) {{
                layer.bindTooltip(feature.properties.statename, {{
                    permanent: true,
                    direction: "center",
                    className: "state-label"
                }});
            }}
        }}).addTo(map);

        function populateDropdown(dropdown, options) {{
            dropdown.innerHTML = "";
            options.forEach(option => {{
                const opt = document.createElement("option");
                opt.value = option;
                opt.textContent = option;
                dropdown.appendChild(opt);
            }});
        }}

        function handleZoom(selectedWards, selectedLGAs, selectedStates) {{
            const bounds = L.latLngBounds([]);
            if (selectedWards.length > 0) {{
                const parentLGAs = new Set();
                wardLayer.eachLayer(layer => {{
                    if (selectedWards.includes(layer.feature.properties.wardname)) {{
                        parentLGAs.add(layer.feature.properties.lganame);
                    }}
                }});
                lgaLayer.eachLayer(layer => {{
                    if (parentLGAs.has(layer.feature.properties.lganame)) {{
                        bounds.extend(layer.getBounds());
                    }}
                }});
            }} else if (selectedLGAs.length > 0 || selectedStates.length > 0) {{
                const parentStates = new Set();
                if (lgaLayer) {{
                    lgaLayer.eachLayer(layer => {{
                        if (selectedLGAs.includes(layer.feature.properties.lganame)) {{
                            parentStates.add(layer.feature.properties.statename);
                        }}
                    }});
                }}
                selectedStates.forEach(state => parentStates.add(state));
                stateLayer.eachLayer(layer => {{
                    if (parentStates.has(layer.feature.properties.statename)) {{
                        bounds.extend(layer.getBounds());
                    }}
                }});
            }}
            if (bounds.isValid()) map.fitBounds(bounds);
        }}

        function handleStateSelection() {{
            const selectedStates = Array.from(document.getElementById("stateDropdown").selectedOptions).map(opt => opt.value);
            populateDropdown(document.getElementById("lgaDropdown"), selectedStates.flatMap(state => stateToLGA[state] || []));
            stateLayer.setStyle(function(feature) {{
                return selectedStates.includes(feature.properties.statename)
                    ? {{ color: "grey", fillColor: "brown", weight: 1, fillOpacity: 0.2 }}
                    : {{ color: "gray", fillColor: "lightgray", weight: 0.5, fillOpacity: 0.1 }};
            }});
            handleZoom([], [], selectedStates);
        }}

        function handleLGASelection() {{
            const selectedLGAs = Array.from(document.getElementById("lgaDropdown").selectedOptions).map(opt => opt.value);
            populateDropdown(document.getElementById("wardDropdown"), selectedLGAs.flatMap(lga => lgaToWard[lga] || []));
            if (lgaLayer) map.removeLayer(lgaLayer);
            lgaLayer = L.geoJSON(lgaGeoJSON, {{
                filter: function(feature) {{ return selectedLGAs.includes(feature.properties.lganame); }},
                style: function() {{ return {{ color: "green", fillColor: "green", weight: 1, fillOpacity: 0.5 }}; }},
                onEachFeature: function(feature, layer) {{
                    layer.bindTooltip(feature.properties.lganame, {{
                        permanent: true,
                        direction: "center",
                        className: "lga-label"
                    }});
                }}
            }}).addTo(map);
            handleZoom([], selectedLGAs, []);
        }}

        function handleWardSelection() {{
            const selectedWards = Array.from(document.getElementById("wardDropdown").selectedOptions).map(opt => opt.value);
            if (wardLayer) map.removeLayer(wardLayer);
            wardLayer = L.geoJSON(wardGeoJSON, {{
                filter: function(feature) {{ return selectedWards.includes(feature.properties.wardname); }},
                style: function() {{ return {{ color: "yellow", fillColor: "yellow", weight: 1, fillOpacity: 0.5 }}; }},
                onEachFeature: function(feature, layer) {{
                    layer.bindTooltip(feature.properties.wardname, {{
                        permanent: true,
                        direction: "center",
                        className: "ward-label"
                    }});
                }}
            }}).addTo(map);
            handleZoom(selectedWards, [], []);
        }}

        function downloadMap() {{
            leafletImage(map, function(err, canvas) {{
                if (err) {{
                    console.error("Error with leaflet-image:", err);
                    downloadMapFallback();
                    return;
                }}
                const link = document.createElement('a');
                link.href = canvas.toDataURL('image/png');
                link.download = 'map.png';
                link.click();
            }});
        }}

        function downloadMapFallback() {{
            const mapContainer = document.getElementById('map-container');
            html2canvas(mapContainer).then(canvas => {{
                const link = document.createElement('a');
                link.href = canvas.toDataURL('image/png');
                link.download = 'map-screenshot.png';
                link.click();
            }}).catch(error => {{
                console.error("Error with html2canvas:", error);
                alert("Unable to download the map.");
            }});
        }}

        // Populate Dropdowns on Page Load
        populateDropdown(document.getElementById("stateDropdown"), Object.keys(stateToLGA));
    </script>
</body>
</html>
"""
# Save the HTML with UTF-8 encoding
with open("interactive_map.html", "w", encoding="utf-8") as f:
    f.write(html_template)

print("Interactive map saved as 'interactive_map.html'")

Interactive map saved as 'interactive_map.html'


# Interactive map 2

In [56]:
html_template = f"""
<!DOCTYPE html>
<html>
<head>
    <meta charset="UTF-8" />
    <meta name="viewport" content="width=device-width, initial-scale=1.0" />
    <title>Interactive Map</title>
    <link rel="stylesheet" href="https://cdn.jsdelivr.net/npm/leaflet@1.9.3/dist/leaflet.css" />
    <link rel="stylesheet" href="https://cdn.jsdelivr.net/npm/bootstrap@5.2.2/dist/css/bootstrap.min.css" />
    <style>
        html, body {
            width: 100%;
            height: 100%;
            margin: 0;
            padding: 0;
            font-family: Arial, sans-serif;
        }
        #container {
            display: flex;
            flex-direction: row;
            width: 100%;
            height: 100%;
        }
        #filters {
            width: 20%;
            padding: 20px;
            background: white;
            border-right: 1px solid #ddd;
            box-shadow: 2px 0 5px rgba(0, 0, 0, 0.1);
            overflow-y: auto;
        }
        #filters select {
            margin-bottom: 15px;
            width: 100%;
            padding: 5px;
        }
        #download-button {
            display: block;
            margin: 20px auto;
            width: 80%;
            background-color: #007bff;
            color: white;
            border: none;
            border-radius: 5px;
            padding: 10px;
            text-align: center;
            cursor: pointer;
        }
        #map-container {
            flex: 1;
            position: relative;
        }
        #map {
            height: 100%;
            width: 100%;
        }
        #search-box {
            position: absolute;
            top: 10px;
            right: 10px;
            z-index: 1000;
            width: 250px;
        }
        .search-suggestions {
            position: absolute;
            top: 50px;
            right: 10px;
            z-index: 1001;
            background: white;
            border: 1px solid #ddd;
            max-height: 200px;
            overflow-y: auto;
            box-shadow: 2px 2px 5px rgba(0, 0, 0, 0.2);
        }
        .search-suggestion {
            padding: 5px;
            cursor: pointer;
        }
        .search-suggestion:hover {
            background-color: #f0f0f0;
        }
    </style>
</head>
<body>
    <div id="container">
        <div id="filters">
            <h5>Filter Map</h5>
            <h6>State</h6>
            <select id="stateDropdown" class="form-select" multiple onchange="handleStateSelection()"></select>
            <h6>Local Government</h6>
            <select id="lgaDropdown" class="form-select" multiple onchange="handleLGASelection()"></select>
            <h6>Ward</h6>
            <select id="wardDropdown" class="form-select" multiple onchange="handleWardSelection()"></select>
            <button id="download-button" onclick="downloadMap()">Download Map</button>
        </div>
        <div id="map-container">
            <div id="search-box">
                <input
                    type="text"
                    id="searchInput"
                    class="form-control"
                    placeholder="Search State, LGA, or Ward"
                    oninput="handleSearch()"
                />
                <div id="searchSuggestions" class="search-suggestions"></div>
            </div>
            <div id="map"></div>
        </div>
    </div>

    <script src="https://cdn.jsdelivr.net/npm/leaflet@1.9.3/dist/leaflet.js"></script>
    <script>
        // Injected GeoJSON data
        const stateGeoJSON = {state_geojson};
        const lgaGeoJSON = {lga_geojson};
        const wardGeoJSON = {ward_geojson};
        const stateToLGA = {json.dumps(state_to_lga)};
        const lgaToWard = {json.dumps(lga_to_ward)};

        // Initialize map
        const map = L.map('map', { zoomControl: true }).setView([9.0820, 8.6753], 6);

        // Add State Layer
        const stateLayer = L.geoJSON(stateGeoJSON, {
            style: {
                color: "gray",
                weight: 1,
                fillOpacity: 0.3,
                fillColor: "beige"
            },
            onEachFeature: (feature, layer) => {
                layer.bindTooltip(feature.properties.statename, {
                    permanent: true,
                    direction: "center"
                });
            }
        }).addTo(map);

        // Add LGA Layer
        const lgaLayer = L.geoJSON(lgaGeoJSON, {
            style: {
                color: "green",
                weight: 1,
                fillOpacity: 0.5,
                fillColor: "lightgreen"
            },
            onEachFeature: (feature, layer) => {
                layer.bindTooltip(feature.properties.lganame, {
                    permanent: false,
                    direction: "center"
                });
            }
        }).addTo(map);

        // Add Ward Layer
        const wardLayer = L.geoJSON(wardGeoJSON, {
            style: {
                color: "red",
                weight: 1,
                fillOpacity: 0.5,
                fillColor: "yellow"
            },
            onEachFeature: (feature, layer) => {
                layer.bindTooltip(feature.properties.wardname, {
                    permanent: false,
                    direction: "center"
                });
            }
        }).addTo(map);

        // Populate Dropdowns
        function populateDropdown(dropdown, options) {
            dropdown.innerHTML = "";
            options.forEach(option => {
                const opt = document.createElement("option");
                opt.value = option;
                opt.textContent = option;
                dropdown.appendChild(opt);
            });
        }

        populateDropdown(document.getElementById("stateDropdown"), Object.keys(stateToLGA));
        populateDropdown(document.getElementById("lgaDropdown"), Object.values(stateToLGA).flat());
        populateDropdown(document.getElementById("wardDropdown"), Object.values(lgaToWard).flat());

        // Handle Dropdown Selection
        function handleStateSelection() {
            const selectedStates = Array.from(document.getElementById("stateDropdown").selectedOptions).map(opt => opt.value);
            const lgaOptions = selectedStates.flatMap(state => stateToLGA[state] || []);
            populateDropdown(document.getElementById("lgaDropdown"), lgaOptions);
        }

        function handleLGASelection() {
            const selectedLGAs = Array.from(document.getElementById("lgaDropdown").selectedOptions).map(opt => opt.value);
            const wardOptions = selectedLGAs.flatMap(lga => lgaToWard[lga] || []);
            populateDropdown(document.getElementById("wardDropdown"), wardOptions);
        }

        // Search Functionality
        function handleSearch() {
            const query = document.getElementById("searchInput").value.toLowerCase();
            const suggestions = [];

            stateLayer.eachLayer(layer => {
                if (layer.feature.properties.statename.toLowerCase().includes(query)) {
                    suggestions.push({ name: layer.feature.properties.statename, layer });
                }
            });

            lgaLayer.eachLayer(layer => {
                if (layer.feature.properties.lganame.toLowerCase().includes(query)) {
                    suggestions.push({ name: layer.feature.properties.lganame, layer });
                }
            });

            wardLayer.eachLayer(layer => {
                if (layer.feature.properties.wardname.toLowerCase().includes(query)) {
                    suggestions.push({ name: layer.feature.properties.wardname, layer });
                }
            });

            renderSearchSuggestions(suggestions);
        }

        function renderSearchSuggestions(suggestions) {
            const suggestionsDiv = document.getElementById("searchSuggestions");
            suggestionsDiv.innerHTML = "";
            suggestions.forEach(({ name, layer }) => {
                const div = document.createElement("div");
                div.className = "search-suggestion";
                div.textContent = name;
                div.onclick = () => {
                    map.fitBounds(layer.getBounds());
                };
                suggestionsDiv.appendChild(div);
            });
        }

        // Download Map as Image
        function downloadMap() {
            leafletImage(map, function(err, canvas) {
                if (err) {
                    console.error("Error generating map image:", err);
                    return;
                }
                const link = document.createElement('a');
                link.href = canvas.toDataURL('image/png');
                link.download = 'map.png';
                link.click();
            });
        }
    </script>
</body>
</html>
"""

# Save the HTML file
with open("interactive_map2.html", "w", encoding="utf-8") as f:
    f.write(html_template)

print("Interactive map saved as 'interactive_map2.html'")

SyntaxError: f-string: expecting '=', or '!', or ':', or '}' (2878050790.py, line 135)